# RNN

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
# from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

In [2]:
data = pd.read_csv('eng_tweets_with_sent_RNN.csv')
data.head()
# Keeping only the neccessary columns
# data = data[['text','sentiment']]

,Company,Name,Retweets,Text,mean_pos,mean_neg,total_pos,total_neg,Sentiment
0,Occipital,Occipital,1 retweet,"Cats out of the bag, we're excited! RT @savvys...",0.069430,0.014881,0,0,1
1,Occipital,Paul White,0 retweets,"No matter what happens, I always have the Occi...",0.075000,0.078125,0,0,-1
2,Occipital,Durlan Castro,0 retweets,Headaches are mostly caused by adhesions in th...,0.086806,0.006944,0,0,1
3,Occipital,a.h. sataN,0 retweets,My occipital spatula is orbiting right now. No...,0.038889,0.036458,0,0,1
4,Occipital,Darin,0 retweets,I wish I could embed my 360's @occipital #SMMOC,0.000000,0.000000,0,0,0


In [3]:
data = data[['Text','Sentiment']]

In [4]:
data

,Text,Sentiment
0,"Cats out of the bag, we're excited! RT @savvys...",1
1,"No matter what happens, I always have the Occi...",-1
2,Headaches are mostly caused by adhesions in th...,1
3,My occipital spatula is orbiting right now. No...,1
4,I wish I could embed my 360's @occipital #SMMOC,0
...,...,...
177936,Last few minutes of #ASRM2012. Make sure to s...,1
177937,Grt @ScottKirsner article on my friend @michel...,1
177938,"@OvaScience, got my seat to tuesday night dinn...",-1
177939,my program director is speaking @ReprodMed 201...,1


In [5]:
data = data[data.Sentiment != 0]
data['Text'] = data['Text'].apply(lambda x: x.lower())
data['Text'] = data['Text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

print(data[ data['Sentiment'] == 1].size)
print(data[ data['Sentiment'] == -1].size)

for idx,row in data.iterrows():
    row[0] = row[0].replace('rt',' ')
    
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['Text'].values)
X = tokenizer.texts_to_sequences(data['Text'].values)
X = pad_sequences(X)

C:\Users\anush\AppData\Local\Temp\ipykernel_11168\2580925338.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Text'] = data['Text'].apply(lambda x: x.lower())
C:\Users\anush\AppData\Local\Temp\ipykernel_11168\2580925338.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Text'] = data['Text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))


175216
84420


In [6]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 32, 128)           256000    
                                                                 
 spatial_dropout1d (SpatialD  (None, 32, 128)          0         
 ropout1D)                                                       
                                                                 
 lstm (LSTM)                 (None, 196)               254800    
                                                                 
 dense (Dense)               (None, 2)                 394       
                                                                 
Total params: 511,194
Trainable params: 511,194
Non-trainable params: 0
_________________________________________________________________
None


In [7]:
Y = pd.get_dummies(data['Sentiment']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(86978, 32) (86978, 2)
(42840, 32) (42840, 2)


In [8]:
batch_size = 32
model.fit(X_train, Y_train, epochs = 50, batch_size=batch_size, verbose = 2)

Epoch 1/50
2719/2719 - 333s - loss: 0.4476 - accuracy: 0.7909 - 333s/epoch - 122ms/step
Epoch 2/50
2719/2719 - 336s - loss: 0.3912 - accuracy: 0.8287 - 336s/epoch - 124ms/step
Epoch 3/50
2719/2719 - 310s - loss: 0.3773 - accuracy: 0.8351 - 310s/epoch - 114ms/step
Epoch 4/50
2719/2719 - 323s - loss: 0.3642 - accuracy: 0.8416 - 323s/epoch - 119ms/step
Epoch 5/50
2719/2719 - 356s - loss: 0.3534 - accuracy: 0.8471 - 356s/epoch - 131ms/step
Epoch 6/50
2719/2719 - 354s - loss: 0.3422 - accuracy: 0.8520 - 354s/epoch - 130ms/step
Epoch 7/50
2719/2719 - 367s - loss: 0.3337 - accuracy: 0.8564 - 367s/epoch - 135ms/step
Epoch 8/50
2719/2719 - 343s - loss: 0.3225 - accuracy: 0.8621 - 343s/epoch - 126ms/step
Epoch 9/50
2719/2719 - 353s - loss: 0.3133 - accuracy: 0.8658 - 353s/epoch - 130ms/step
Epoch 10/50
2719/2719 - 320s - loss: 0.3042 - accuracy: 0.8706 - 320s/epoch - 118ms/step
Epoch 11/50
2719/2719 - 320s - loss: 0.2947 - accuracy: 0.8747 - 320s/epoch - 118ms/step
Epoch 12/50
2719/2719 - 319s -

In [9]:
validation_size = 1500

X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

1292/1292 - 28s - loss: 0.6511 - accuracy: 0.8125 - 28s/epoch - 22ms/step
score: 0.65
acc: 0.81


In [10]:
pos_cnt, neg_cnt, pos_correct, neg_correct = 0, 0, 0, 0
for x in range(len(X_validate)):
    
    result = model.predict(X_validate[x].reshape(1,X_test.shape[1]),batch_size=1,verbose = 2)[0]
   
    if np.argmax(result) == np.argmax(Y_validate[x]):
        if np.argmax(Y_validate[x]) == 0:
            neg_correct += 1
        else:
            pos_correct += 1
       
    if np.argmax(Y_validate[x]) == 0:
        neg_cnt += 1
    else:
        pos_cnt += 1
    
 


print("pos_acc", pos_correct/pos_cnt*100, "%")
print("neg_acc", neg_correct/neg_cnt*100, "%")

1/1 - 1s - 577ms/epoch - 577ms/step
1/1 - 0s - 38ms/epoch - 38ms/step
1/1 - 0s - 40ms/epoch - 40ms/step
1/1 - 0s - 40ms/epoch - 40ms/step
1/1 - 0s - 38ms/epoch - 38ms/step
1/1 - 0s - 37ms/epoch - 37ms/step
1/1 - 0s - 41ms/epoch - 41ms/step
1/1 - 0s - 39ms/epoch - 39ms/step
1/1 - 0s - 40ms/epoch - 40ms/step
1/1 - 0s - 39ms/epoch - 39ms/step
1/1 - 0s - 40ms/epoch - 40ms/step
1/1 - 0s - 39ms/epoch - 39ms/step
1/1 - 0s - 39ms/epoch - 39ms/step
1/1 - 0s - 41ms/epoch - 41ms/step
1/1 - 0s - 43ms/epoch - 43ms/step
1/1 - 0s - 39ms/epoch - 39ms/step
1/1 - 0s - 40ms/epoch - 40ms/step
1/1 - 0s - 42ms/epoch - 42ms/step
1/1 - 0s - 43ms/epoch - 43ms/step
1/1 - 0s - 42ms/epoch - 42ms/step
1/1 - 0s - 39ms/epoch - 39ms/step
1/1 - 0s - 41ms/epoch - 41ms/step
1/1 - 0s - 40ms/epoch - 40ms/step
1/1 - 0s - 40ms/epoch - 40ms/step
1/1 - 0s - 41ms/epoch - 41ms/step
1/1 - 0s - 42ms/epoch - 42ms/step
1/1 - 0s - 40ms/epoch - 40ms/step
1/1 - 0s - 39ms/epoch - 39ms/step
1/1 - 0s - 42ms/epoch - 42ms/step
1/1 - 0s - 4

1/1 - 0s - 39ms/epoch - 39ms/step
1/1 - 0s - 43ms/epoch - 43ms/step
1/1 - 0s - 43ms/epoch - 43ms/step
1/1 - 0s - 43ms/epoch - 43ms/step
1/1 - 0s - 38ms/epoch - 38ms/step
1/1 - 0s - 44ms/epoch - 44ms/step
1/1 - 0s - 41ms/epoch - 41ms/step
1/1 - 0s - 40ms/epoch - 40ms/step
1/1 - 0s - 43ms/epoch - 43ms/step
1/1 - 0s - 40ms/epoch - 40ms/step
1/1 - 0s - 40ms/epoch - 40ms/step
1/1 - 0s - 41ms/epoch - 41ms/step
1/1 - 0s - 43ms/epoch - 43ms/step
1/1 - 0s - 39ms/epoch - 39ms/step
1/1 - 0s - 42ms/epoch - 42ms/step
1/1 - 0s - 45ms/epoch - 45ms/step
1/1 - 0s - 42ms/epoch - 42ms/step
1/1 - 0s - 50ms/epoch - 50ms/step
1/1 - 0s - 40ms/epoch - 40ms/step
1/1 - 0s - 40ms/epoch - 40ms/step
1/1 - 0s - 39ms/epoch - 39ms/step
1/1 - 0s - 39ms/epoch - 39ms/step
1/1 - 0s - 40ms/epoch - 40ms/step
1/1 - 0s - 40ms/epoch - 40ms/step
1/1 - 0s - 43ms/epoch - 43ms/step
1/1 - 0s - 41ms/epoch - 41ms/step
1/1 - 0s - 39ms/epoch - 39ms/step
1/1 - 0s - 39ms/epoch - 39ms/step
1/1 - 0s - 38ms/epoch - 38ms/step
1/1 - 0s - 43m

1/1 - 0s - 41ms/epoch - 41ms/step
1/1 - 0s - 42ms/epoch - 42ms/step
1/1 - 0s - 39ms/epoch - 39ms/step
1/1 - 0s - 41ms/epoch - 41ms/step
1/1 - 0s - 42ms/epoch - 42ms/step
1/1 - 0s - 41ms/epoch - 41ms/step
1/1 - 0s - 41ms/epoch - 41ms/step
1/1 - 0s - 42ms/epoch - 42ms/step
1/1 - 0s - 42ms/epoch - 42ms/step
1/1 - 0s - 40ms/epoch - 40ms/step
1/1 - 0s - 40ms/epoch - 40ms/step
1/1 - 0s - 43ms/epoch - 43ms/step
1/1 - 0s - 44ms/epoch - 44ms/step
1/1 - 0s - 39ms/epoch - 39ms/step
1/1 - 0s - 40ms/epoch - 40ms/step
1/1 - 0s - 42ms/epoch - 42ms/step
1/1 - 0s - 41ms/epoch - 41ms/step
1/1 - 0s - 42ms/epoch - 42ms/step
1/1 - 0s - 41ms/epoch - 41ms/step
1/1 - 0s - 42ms/epoch - 42ms/step
1/1 - 0s - 40ms/epoch - 40ms/step
1/1 - 0s - 41ms/epoch - 41ms/step
1/1 - 0s - 41ms/epoch - 41ms/step
1/1 - 0s - 41ms/epoch - 41ms/step
1/1 - 0s - 41ms/epoch - 41ms/step
1/1 - 0s - 40ms/epoch - 40ms/step
1/1 - 0s - 41ms/epoch - 41ms/step
1/1 - 0s - 41ms/epoch - 41ms/step
1/1 - 0s - 39ms/epoch - 39ms/step
1/1 - 0s - 46m

1/1 - 0s - 40ms/epoch - 40ms/step
1/1 - 0s - 41ms/epoch - 41ms/step
1/1 - 0s - 44ms/epoch - 44ms/step
1/1 - 0s - 42ms/epoch - 42ms/step
1/1 - 0s - 41ms/epoch - 41ms/step
1/1 - 0s - 42ms/epoch - 42ms/step
1/1 - 0s - 43ms/epoch - 43ms/step
1/1 - 0s - 41ms/epoch - 41ms/step
1/1 - 0s - 43ms/epoch - 43ms/step
1/1 - 0s - 44ms/epoch - 44ms/step
1/1 - 0s - 41ms/epoch - 41ms/step
1/1 - 0s - 42ms/epoch - 42ms/step
1/1 - 0s - 43ms/epoch - 43ms/step
1/1 - 0s - 40ms/epoch - 40ms/step
1/1 - 0s - 41ms/epoch - 41ms/step
1/1 - 0s - 45ms/epoch - 45ms/step
1/1 - 0s - 46ms/epoch - 46ms/step
1/1 - 0s - 42ms/epoch - 42ms/step
1/1 - 0s - 43ms/epoch - 43ms/step
1/1 - 0s - 42ms/epoch - 42ms/step
1/1 - 0s - 42ms/epoch - 42ms/step
1/1 - 0s - 42ms/epoch - 42ms/step
1/1 - 0s - 42ms/epoch - 42ms/step
1/1 - 0s - 42ms/epoch - 42ms/step
1/1 - 0s - 41ms/epoch - 41ms/step
1/1 - 0s - 42ms/epoch - 42ms/step
1/1 - 0s - 43ms/epoch - 43ms/step
1/1 - 0s - 44ms/epoch - 44ms/step
1/1 - 0s - 43ms/epoch - 43ms/step
1/1 - 0s - 42m

1/1 - 0s - 47ms/epoch - 47ms/step
1/1 - 0s - 45ms/epoch - 45ms/step
1/1 - 0s - 43ms/epoch - 43ms/step
1/1 - 0s - 43ms/epoch - 43ms/step
1/1 - 0s - 45ms/epoch - 45ms/step
1/1 - 0s - 44ms/epoch - 44ms/step
1/1 - 0s - 44ms/epoch - 44ms/step
1/1 - 0s - 46ms/epoch - 46ms/step
1/1 - 0s - 44ms/epoch - 44ms/step
1/1 - 0s - 42ms/epoch - 42ms/step
1/1 - 0s - 42ms/epoch - 42ms/step
1/1 - 0s - 43ms/epoch - 43ms/step
1/1 - 0s - 46ms/epoch - 46ms/step
1/1 - 0s - 44ms/epoch - 44ms/step
1/1 - 0s - 44ms/epoch - 44ms/step
1/1 - 0s - 46ms/epoch - 46ms/step
1/1 - 0s - 43ms/epoch - 43ms/step
1/1 - 0s - 46ms/epoch - 46ms/step
1/1 - 0s - 44ms/epoch - 44ms/step
1/1 - 0s - 45ms/epoch - 45ms/step
1/1 - 0s - 45ms/epoch - 45ms/step
1/1 - 0s - 43ms/epoch - 43ms/step
1/1 - 0s - 42ms/epoch - 42ms/step
1/1 - 0s - 43ms/epoch - 43ms/step
1/1 - 0s - 44ms/epoch - 44ms/step
1/1 - 0s - 49ms/epoch - 49ms/step
1/1 - 0s - 44ms/epoch - 44ms/step
1/1 - 0s - 44ms/epoch - 44ms/step
1/1 - 0s - 44ms/epoch - 44ms/step
1/1 - 0s - 41m

1/1 - 0s - 47ms/epoch - 47ms/step
1/1 - 0s - 47ms/epoch - 47ms/step
1/1 - 0s - 45ms/epoch - 45ms/step
1/1 - 0s - 47ms/epoch - 47ms/step
1/1 - 0s - 47ms/epoch - 47ms/step
1/1 - 0s - 44ms/epoch - 44ms/step
1/1 - 0s - 47ms/epoch - 47ms/step
1/1 - 0s - 46ms/epoch - 46ms/step
1/1 - 0s - 47ms/epoch - 47ms/step
1/1 - 0s - 47ms/epoch - 47ms/step
1/1 - 0s - 47ms/epoch - 47ms/step
1/1 - 0s - 46ms/epoch - 46ms/step
1/1 - 0s - 46ms/epoch - 46ms/step
1/1 - 0s - 44ms/epoch - 44ms/step
1/1 - 0s - 47ms/epoch - 47ms/step
1/1 - 0s - 46ms/epoch - 46ms/step
1/1 - 0s - 46ms/epoch - 46ms/step
1/1 - 0s - 46ms/epoch - 46ms/step
1/1 - 0s - 47ms/epoch - 47ms/step
1/1 - 0s - 45ms/epoch - 45ms/step
1/1 - 0s - 45ms/epoch - 45ms/step
1/1 - 0s - 47ms/epoch - 47ms/step
1/1 - 0s - 45ms/epoch - 45ms/step
1/1 - 0s - 46ms/epoch - 46ms/step
1/1 - 0s - 47ms/epoch - 47ms/step
1/1 - 0s - 46ms/epoch - 46ms/step
1/1 - 0s - 44ms/epoch - 44ms/step
1/1 - 0s - 47ms/epoch - 47ms/step
1/1 - 0s - 48ms/epoch - 48ms/step
1/1 - 0s - 46m

1/1 - 0s - 51ms/epoch - 51ms/step
1/1 - 0s - 48ms/epoch - 48ms/step
1/1 - 0s - 48ms/epoch - 48ms/step
1/1 - 0s - 48ms/epoch - 48ms/step
1/1 - 0s - 48ms/epoch - 48ms/step
1/1 - 0s - 49ms/epoch - 49ms/step
1/1 - 0s - 48ms/epoch - 48ms/step
1/1 - 0s - 48ms/epoch - 48ms/step
1/1 - 0s - 49ms/epoch - 49ms/step
1/1 - 0s - 49ms/epoch - 49ms/step
1/1 - 0s - 49ms/epoch - 49ms/step
1/1 - 0s - 48ms/epoch - 48ms/step
1/1 - 0s - 55ms/epoch - 55ms/step
1/1 - 0s - 52ms/epoch - 52ms/step
1/1 - 0s - 49ms/epoch - 49ms/step
1/1 - 0s - 50ms/epoch - 50ms/step
1/1 - 0s - 50ms/epoch - 50ms/step
1/1 - 0s - 50ms/epoch - 50ms/step
1/1 - 0s - 51ms/epoch - 51ms/step
1/1 - 0s - 49ms/epoch - 49ms/step
1/1 - 0s - 48ms/epoch - 48ms/step
1/1 - 0s - 50ms/epoch - 50ms/step
1/1 - 0s - 50ms/epoch - 50ms/step
1/1 - 0s - 49ms/epoch - 49ms/step
1/1 - 0s - 49ms/epoch - 49ms/step
1/1 - 0s - 48ms/epoch - 48ms/step
1/1 - 0s - 54ms/epoch - 54ms/step
1/1 - 0s - 49ms/epoch - 49ms/step
1/1 - 0s - 47ms/epoch - 47ms/step
1/1 - 0s - 50m